In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from openpyxl import Workbook

# Tarayıcı başlatma
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Temel URL ve değişken kısımlar
base_url = "https://www.amazon.com.tr/s?k=kitap"
url_parts = [
    "&qid=1720685438&ref=sr_pg_1",
    "&page=2&qid=1720715811&ref=sr_pg_2",
    "&page=3&qid=1720716483&ref=sr_pg_3",
    "&page=4&qid=1720716514&ref=sr_pg_4",
    "&page=5&qid=1720716530&ref=sr_pg_5",
    "&page=6&qid=1720716543&ref=sr_pg_6",
    "&page=7&qid=1720716564&ref=sr_pg_7"
]

# Excel dosyası oluşturma
wb = Workbook()
sheet = wb.active
sheet.title = "Amazon Kitap Bestseller"
sheet.append(["Kitap", "Yazar", "Yıldız", "Değerlendiren Kişi Sayısı", "Fiyat"])

for part in url_parts:
    url = base_url + part
    driver.get(url)
    time.sleep(5)
    
    books = driver.find_elements(By.CSS_SELECTOR, "div.s-main-slot div.s-result-item")

    for book in books:
        book_title = book.find_element(By.CSS_SELECTOR, "h2 span").text if book.find_elements(By.CSS_SELECTOR, "h2 span") else ""
        author = book.find_element(By.CSS_SELECTOR, "div.a-row.a-size-base.a-color-secondary").text if book.find_elements(By.CSS_SELECTOR, "div.a-row.a-size-base.a-color-secondary") else ""
        stars = book.find_element(By.CSS_SELECTOR, "span.a-icon-alt").get_attribute("innerHTML") if book.find_elements(By.CSS_SELECTOR, "span.a-icon-alt") else ""
        rating_count = book.find_element(By.CSS_SELECTOR, "div.a-row.a-size-small span.a-size-base").text if book.find_elements(By.CSS_SELECTOR, "div.a-row.a-size-small span.a-size-base") else ""
        price_whole = book.find_element(By.CLASS_NAME, "a-price-whole").text if book.find_elements(By.CLASS_NAME, "a-price-whole") else ""
        price_fraction = book.find_element(By.CLASS_NAME, "a-price-fraction").text if book.find_elements(By.CLASS_NAME, "a-price-fraction") else "00"
        price = f"{price_whole},{price_fraction} TL" if price_whole else ""

        # Yıldız bilgisi bulunamazsa kaydetme
        if book_title and author and stars and rating_count and price:
            print(f"Kitap: {book_title}, Yazar: {author}, Yıldız: {stars}, Değerlendiren Kişi Sayısı: {rating_count}, Fiyat: {price}")
            sheet.append([book_title, author, stars, rating_count, price])
    
    time.sleep(2)

excel_file_path = r'C:\Users\ilayd\OneDrive\Masaüstü\Yeni klasör\amazon_bestsellers.xlsx'
wb.save(excel_file_path)
driver.quit()
print(f"Veriler başarıyla '{excel_file_path}' dosyasına kaydedildi.")        